In [50]:
from dataclasses import dataclass, field
from heapq import heappush, heappop
from pprint import pprint # pretty-printing basic data structures

import pandas as pd

CLOCK = 0

Clock

In [37]:
# https://stackoverflow.com/questions/279561/what-is-the-python-equivalent-of-static-variables-inside-a-function
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

@static_vars(t=0)
def now():
    return now.t

def set_time(t_new=0):
    now.t = t_new
    return now()

print("The current simulation time is", now(), "o'clock.")

The current simulation time is 0 o'clock.


Events

In [10]:
@dataclass
class Event:
    name: str

@dataclass(order=True)
class EventInstance:
    name: str=field(compare=False)
    parent: Event=field(compare=False)
    time: float

Activities

In [13]:
@dataclass
class Activity:
    name: str
    start: Event
    end: Event
    duration: float

# waiting = Activity("waiting", INIT, liftoff, 0)
# s1_ascent = Activity("s1_ascent", liftoff, stage, 5)
# s2_ascent = Activity("s2_ascent", stage, orbit, 5)

Future Event List

In [67]:
# Implement the FutureEventList from Prof. Vuduc's example
class FutureEventList:
    def __init__(self):
        self.events = []
        
    def __iter__(self):
        return self
    
    def __next__(self) -> Event:
        from heapq import heappop
        if self.events:
            return heappop(self.events)
        raise StopIteration
    
    def __repr__(self) -> str:
        from pprint import pformat
        return pformat(self.events)

    def schedule(self, activity: Activity) -> None:
        heappush(
            self.events,
            EventInstance(activity.end.name, activity.end, now() + activity.duration)
        )

    def get_next(self):
        return heappop(self.events)

Vehicles

In [68]:
@dataclass
class Vehicle:
    name: str
    activity: Activity
    propload: float
    trace: pd.DataFrame = pd.DataFrame(columns=['CurrentEvent', 'NextEvent', 'Prop', 'Activity'])

def start_vehicle(name, conops, propload = 100):
    v = Vehicle(name, conops.first(), propload)
    v.trace = pd.concat([
        v.trace,
        pd.DataFrame({"CurrentEvent": v.activity.start, "NextEvent": v.activity.end, "Prop": v.propload, "Activity": v.activity}, index = [len(v.trace) + 1])
    ])
    return v

### Process Pseudocode

In [69]:


@dataclass
class ConOps:
    sequence: dict

    def first(self):
        return self.sequence["INIT"]

    def after(self, current_event):
        # Get the activity which starts with a particular event
        return self.sequence[current_event.name]

First define the ConOps as a sequence of processes

In [74]:
# Events
INIT = Event("INIT")
liftoff = Event("Liftoff")
stage = Event("Stage")
orbit = Event("Orbit")

# Activities
activities = {
    INIT.name: Activity("waiting", INIT, liftoff, 0),
    liftoff.name: Activity("s1_ascent", liftoff, stage, 10),
    stage.name: Activity("s2_ascent", stage, orbit, 10)
}

conops = ConOps(activities)

Initialize the Simulation

In [75]:
# Reset the clock
set_time(0)

# Create and empty events list
future = FutureEventList()

# Start a vehicle
v = start_vehicle("LV1", conops)

# Vehicle starts in some activity, which will end when that activities event is processed
current_activity = v.activity
future.schedule(v.activity)

Walk the events

In [76]:
# Trigger the ending event
current_event = future.get_next()

# Process the event
# - Update the system state
# - Update the vehicle state
# v.propload -= 60  # some fancy code to handle this

# Get the next activity
next_activity = conops.after(current_event)

print(f"The current activity is {current_activity}")
print(f"The current event is {current_event}")
print(f"The next activity is {next_activity}")

# - Change the activity
v.activity = next_activity

# Schedule the next event
future.schedule(v.activity)
future

assert current_event.parent == next_activity.start

# - Update the vehicle trace
v.trace = pd.concat([
    v.trace,
    pd.DataFrame({"CurrentEvent": v.activity.start, "NextEvent": v.activity.end, "Prop": v.propload, "Activity": v.activity}, index = [len(v.trace) + 1])
])

v.trace

The current activity is Activity(name='waiting', start=Event(name='INIT'), end=Event(name='Liftoff'), duration=0)
The current event is EventInstance(name='Liftoff', parent=Event(name='Liftoff'), time=0)
The next activity is Activity(name='s1_ascent', start=Event(name='Liftoff'), end=Event(name='Stage'), duration=10)


,CurrentEvent,NextEvent,Prop,Activity
1,Event(name='INIT'),Event(name='Liftoff'),100,"Activity(name='waiting', start=Event(name='INI..."
2,Event(name='Liftoff'),Event(name='Stage'),100,"Activity(name='s1_ascent', start=Event(name='L..."


In [8]:
# activities = {
#     "waiting": [v],
#     "s1_ascent": [],
#     "s2_Ascent": [],
#     "done": [],
# }

# # - Update the system state
# activities[prev_activity.name].remove(v)
# activities[next_activity.name].append(v)